# HadesMiMC

In [1]:
import itertools
def compose_permutations(P1, P2, n): # return polytope for F2 \circ F1
    ieqs = []
    eqns = []
    for ieq in P1.inequalities_list():
        ieqs.append(ieq[:2*n+1] + [0]*n + ieq[-1:] + [0])
    for eqn in P1.equations_list():
        eqns.append(eqn[:2*n+1] + [0]*n + eqn[-1:] + [0])
    for ieq in P2.inequalities_list():
        ieqs.append(ieq[:1] + [0]*n + ieq[1:2*n+1] + [0] + ieq[-1:])
    for eqn in P2.equations_list():
        eqns.append(eqn[:1] + [0]*n + eqn[1:2*n+1] + [0] + eqn[-1:])
    M = [[matrix.zero(ZZ, i, j) for j in (n, n, n, 1, 1)] for i in (n, n, 1)]
    M[0][0] = matrix.identity(ZZ, n)
    M[1][2] = matrix.identity(ZZ, n)
    M[2][3] = matrix.identity(ZZ, 1)
    M[2][4] = matrix.identity(ZZ, 1)
    return Polyhedron(ieqs=ieqs, eqns = eqns).linear_transformation(block_matrix(M))

def reduce_to_single_variable(P, n):
    return P.linear_transformation(Matrix(ZZ, [[1]*n + [0]*n + [0], [0]*n + [1]*n + [0], [0]*n + [0]*n + [1]]))

def reduced_parallel_application(Ps, ns):
    return sum(reduce_to_single_variable(P, n) if n > 1 else P for P, n in zip(Ps, ns))

def get_polytope_function(p, n, d): # model could still be improved when d < n-1 as the degree influences the results then
    ieqs = []
    for i in range(2*n):
        ieqs.append([0] + [0]*i + [1] + [0]*(2*n-i))
        ieqs.append([p-1] + [0]*i + [-1] + [0]*(2*n-i))
    ieqs.append([0]*(2*n+1) + [1]) # ordp is at least 0
    ieqs.append([0] + [-1]*n + [d]*n + [d*(p-1)]) # constraints from function
    ieqs.append([-1] + [0]*n + [1]*n + [0]) # special treatment for output character 1
    return Polyhedron(ieqs=ieqs).convex_hull(Polyhedron(vertices=[[0]*(2*n+1)]))
    
def get_polytope_permutation(p, n, d): # model could still be improved when d < n-1 as the degree influences the results then
    ieqs = []
    for i in range(2*n):
        ieqs.append([0] + [0]*i + [1] + [0]*(2*n-i))
        ieqs.append([p-1] + [0]*i + [-1] + [0]*(2*n-i))
    ieqs.append([0]*(2*n+1) + [1]) # ord_p is at least 0
    ieqs.append([0] + [-1]*n + [d]*n + [d*(p-1)]) # constraints from function
    ieqs.append([-1] + [0]*n + [1]*n + [0]) # special treatment for output character 1
    ieqs.append([(p-1)*n-1] + [-1]*n + [0]*n + [0]) # special treatment for input character of maximal wheight
    P1 = Polyhedron(ieqs=ieqs)
    # second polytope for maximal wheight input character
    ieqs = []
    eqns = []
    for i in range(n):
        ieqs.append([0] + [0]*n + [0]*i + [1] + [0]*(n-i))
        ieqs.append([p-1] + [0]*n + [0]*i + [-1] + [0]*(n-i))
        eqns.append([p-1] + [0]*i + [-1] + [0]*(n-1-i) + [0]*(n+1))
    ieqs.append([-1] + [0]*(2*n) + [1]) # ordp is at least 1 in this case
    ieqs.append([-1] + [0]*n + [1]*n + [0]) # special treatment for output character 1
    ieqs.append([(p-1)*n-1] + [0]*n + [-1]*n + [0]) # special treatment for output character of maximal wheight
    P2 = Polyhedron(ieqs=ieqs,eqns=eqns)
    # missing points in third polytope
    P3 = Polyhedron(vertices=[[0]*(2*n+1), [p-1]*2*n + [0]])
    return P1.convex_hull(P2).convex_hull(P3)

def get_polytope_affine_power_permutation(p, d): # returns a polytope for the function (x+a)^d
    _ds = [p-1, d]
    while _ds[-1] != 1:
        _ds.append(_ds[-2] % _ds[-1])
    dinv = pow(d, -1, p-1)
    _costs = [p-1] + [(x*dinv) % (p-1) for x in _ds[1:]]
    ds, costs = _ds[-1:], _costs[-1:]
    for i in range(len(_ds)-2, 0, -1):
        if _ds[i] * costs[-1] > ds[-1]*_costs[i]:
            ds.append(_ds[i])
            costs.append(_costs[i])
    ds = ds[::-1]
    costs = costs[::-1]

    # and now for the curve
    points = [[0, 0, 0]]
    leftover = p-1
    traded = 0
    for d, c in zip(ds, costs):
        x = min(leftover // c, (p-1-traded) // d)
        leftover -= x*c
        traded += x*d
        points.append([traded, p-1-leftover, 0])
    P = Polyhedron(vertices = points)
    Paffine = get_polytope_permutation(p, 1, 1)
    P2 = compose_permutations(Paffine, P, 1)
    return P2

def get_polytope_reduced_linear_permutation(p, n):
    ieqs = []
    for i in range(2):
        ieqs.append([0] + [0]*i + [1] + [0]*(2-i))
        ieqs.append([n*(p-1)] + [0]*i + [-1] + [0]*(2-i))
    ieqs.append([0, 0, 0, 1]) # ord_p is at least 0
    ieqs.append([0, -1, 1, p-1]) # constraints from function
    ieqs.append([-1, 0, 1, 0]) # special treatment for output character 1
    ieqs.append([(p-1)*n-1, -1, 0, 0]) # special treatment for input character of maximal wheight
    P1 = Polyhedron(ieqs=ieqs)
    # second polytope for maximal wheight input character
    ieqs = []
    eqns = []
    ieqs.append([0, 0, 1, 0])
    ieqs.append([n*(p-1), 0, -1, 0])
    eqns.append([p-1, -1, 0, 0])
    ieqs.append([-1, 0, 0, 1]) # ordp is at least 1 in this case
    ieqs.append([-1, 0, 1, 0]) # special treatment for output character 1
    ieqs.append([(p-1)*n-1, 0, -1, 0]) # special treatment for output character of maximal wheight
    P2 = Polyhedron(ieqs=ieqs,eqns=eqns)
    # missing points in third polytope
    P3 = Polyhedron(vertices=[[0]*3, [n*(p-1), n*(p-1), 0]])
    return P1.convex_hull(P2).convex_hull(P3)

In [2]:
def constraint_variables_in_polyhedron(model, P, vrs):
    assert(P.ambient_dimension() == len(vrs))
    vrs = (1, ) + tuple(vrs)
    for ieq in P.inequalities_list():
        model.add_constraint(sum(x*y for x, y in zip(vrs, ieq)) >= 0)
    for eqn in P.equations_list():
        model.add_constraint(sum(x*y for x, y in zip(vrs, eqn)) == 0)

## Generic SPN

### 64-bit prime

In [3]:
# degrees mod p
p = 2**64-2**32+1
d = 7
t = 8

P_cell = get_polytope_affine_power_permutation(p, d)
P_round = compose_permutations(t*P_cell, get_polytope_reduced_linear_permutation(p, t), 1)
P_full = P_round
i = 1
degrees = []
while True:
    model, vrs = P_full.to_linear_program(solver="PPL", return_variable=True)
    model.set_integer(vrs)
    model.set_min(vrs, 0)
    model.set_max(vrs, t*(p-1))
    model.set_objective(vrs[0])
    model.add_constraint(vrs[1] == 1)
    model.add_constraint(vrs[2] <= 0)
    res = model.solve()
    print(i,res)
    degrees.append(res)
    if res >= t*(p-1)-1:
        break
    i += 1
    P_full = compose_permutations(P_full, P_round, 1)

1 7
2 49
3 343
4 2401
5 16807
6 117649
7 823543
8 5764801
9 40353607
10 282475249
11 1977326743
12 13841287201
13 96889010407
14 678223072849
15 4747561509943
16 33232930569601
17 232630513987207
18 1628413597910449
19 11398895185373143
20 79792266297612001
21 558545864083284007
22 3909821048582988049
23 27368747340080916343
24 147573952555316674524
25 147573952555316674559


In [4]:
# properties mod p based on degree
pdiv = divisors(p-1)[:-1]
for r, deg in enumerate(degrees, start=1):
    cells, subgroup = 0, 0
    cells = floor(deg/(p-1))
    deg -= cells*(p-1)
    if deg >= pdiv[-1]:
        cells += 1
    else:
        for subgroup in pdiv:
            if subgroup > deg:
                break
    print(r, cells, subgroup, f"{subgroup+1}*p^{cells}")

1 0 8 9*p^0
2 0 51 52*p^0
3 0 384 385*p^0
4 0 2560 2561*p^0
5 0 17408 17409*p^0
6 0 122880 122881*p^0
7 0 835584 835585*p^0
8 0 6291456 6291457*p^0
9 0 41943040 41943041*p^0
10 0 285212672 285212673*p^0
11 0 2013265920 2013265921*p^0
12 0 16106127360 16106127361*p^0
13 0 103080787968 103080787969*p^0
14 0 687205253120 687205253121*p^0
15 0 5497642024960 5497642024961*p^0
16 0 35047467909120 35047467909121*p^0
17 0 263886817198080 263886817198081*p^0
18 0 1695472800497664 1695472800497665*p^0
19 0 11962869046312960 11962869046312961*p^0
20 0 90425216026542080 90425216026542081*p^0
21 0 576460752169205760 576460752169205761*p^0
22 0 4611686017353646080 4611686017353646081*p^0
23 1 9223372034707292160 9223372034707292161*p^1
24 8 0 1*p^8
25 8 0 1*p^8


In [5]:
# properties mod p as if SHARK-like
pdiv = divisors(p-1)[:-1]
for r, deg in enumerate(degrees, start=2):
    cells, subgroup = 0, 0
    cells = floor(deg/(p-1))
    deg -= cells*(p-1)
    # extend by one round (for one cell)
    if deg > 0:
        model, vrs = P_cell.to_linear_program(solver="PPL", return_variable=True)
        model.set_integer(vrs)
        model.set_min(vrs, 0)
        model.set_max(vrs, p-1)
        model.set_objective(vrs[0])
        model.add_constraint(vrs[1] == deg)
        model.add_constraint(vrs[2] <= 0)
        new_deg = model.solve()
        if new_deg >= pdiv[-1]:
            cells += 1
        else:
            for subgroup in pdiv:
                if subgroup > new_deg:
                    break
    print(r, cells, subgroup, f"{subgroup+1}*p^{cells}")

2 0 51 52*p^0
3 0 384 385*p^0
4 0 2560 2561*p^0
5 0 17408 17409*p^0
6 0 122880 122881*p^0
7 0 835584 835585*p^0
8 0 6291456 6291457*p^0
9 0 41943040 41943041*p^0
10 0 285212672 285212673*p^0
11 0 2013265920 2013265921*p^0
12 0 16106127360 16106127361*p^0
13 0 103080787968 103080787969*p^0
14 0 687205253120 687205253121*p^0
15 0 5497642024960 5497642024961*p^0
16 0 35047467909120 35047467909121*p^0
17 0 263886817198080 263886817198081*p^0
18 0 1695472800497664 1695472800497665*p^0
19 0 11962869046312960 11962869046312961*p^0
20 0 90425216026542080 90425216026542081*p^0
21 0 576460752169205760 576460752169205761*p^0
22 0 4611686017353646080 4611686017353646081*p^0
23 1 0 1*p^1
24 2 0 1*p^2
25 8 0 1*p^8
26 8 0 1*p^8


In [6]:
# degrees mod p^2
p = 2**64-2**32+1
d = 7
t = 8

P_cell = get_polytope_affine_power_permutation(p, d)
P_round = compose_permutations(t*P_cell, get_polytope_reduced_linear_permutation(p, t), 1)
P_full = P_round
i = 1
degrees_p2 = []
while True:
    model, vrs = P_full.to_linear_program(solver="PPL", return_variable=True)
    model.set_integer(vrs)
    model.set_min(vrs, 0)
    model.set_max(vrs, t*(p-1))
    model.set_objective(vrs[0])
    model.add_constraint(vrs[1] == 1)
    model.add_constraint(vrs[2] <= 1)
    res = model.solve()
    if res != d**i:
        print(i,res)
    degrees_p2.append(res)
    if res >= t*(p-1)-1:
        break
    i += 1
    P_full = compose_permutations(P_full, P_round, 1)

1 129127208485902090247
2 147573952555316674560


In [7]:
# properties mod p^2 based on degree
pdiv = divisors(p-1)[:-1]
for r, deg in enumerate(degrees_p2, start=1):
    cells, subgroup = 0, 0
    cells = floor(deg/(p-1))
    deg -= cells*(p-1)
    if deg >= pdiv[-1]:
        cells += 1
    else:
        for subgroup in pdiv:
            if subgroup > deg:
                break
    print(r, cells, subgroup, f"{subgroup+1}*p^{cells}")

1 7 8 9*p^7
2 8 1 2*p^8


In [8]:
# SHARK-like properties with higher divisibility based on saturating sboxes
p = 2**64-2**32+1
d = 7
t = 8

P_cell = get_polytope_affine_power_permutation(p, d)
P_round = compose_permutations(t*P_cell, get_polytope_reduced_linear_permutation(p, t), 1)
P_full = get_polytope_reduced_linear_permutation(p, t)
i = 1
degrees_p2 = []
while True:
    for cells in range(1, t):
        Pred = (P_full & Polyhedron(eqns = [[-1, 0, 1, 0], [-(p-1)*cells, 1, 0, (p-1)]])).linear_transformation(Matrix(ZZ, 1, 3, [[0, 0, 1]]))
        res = min(ceil(x[0]) for x in Pred.vertices_list())
        if res >= 2:
            print(i, cells, res)
    if cells == t-1 and res <= 1:
        break
    i += 1
    P_full = compose_permutations(P_full, P_round, 1)

1 3 2
1 4 2
1 5 3
1 6 3
1 7 4


### Degree 2 extension field over 31-bit prime


In [9]:
# q-degrees mod p
p = 2**31-2**24+1
e = 2
d = 7
t = 8

P_cell = get_polytope_affine_power_permutation(p**e, d)
P_round = compose_permutations(t*P_cell, get_polytope_reduced_linear_permutation(p**e, t), 1)
P_full = P_round
r = 1
qdegrees = []
while True:
    model = MixedIntegerLinearProgram('PPL', maximization=True)
    digits_input_exponents = model.new_variable(integer=True, nonnegative=True)
    model.set_max(digits_input_exponents, t*(p-1))
    rotations_input_exponents = [sum(p**i*digits_input_exponents[(i-j) % e] for i in range(e)) for j in range(e)]
    valuations = model.new_variable(integer=True, nonnegative=True)
    valuation = 0
    for i in range(e):
        valuation += valuations[i]
        constraint_variables_in_polyhedron(model, P_full, [rotations_input_exponents[i], p**i, valuations[i]])
    model.add_constraint(valuation <= 0)
    model.set_objective(rotations_input_exponents[0])
    res = model.solve()
    print(r, res)
    qdegrees.append(res)
    if res >= t*(p**e-1)-1:
        break
    r += 1
    P_full = compose_permutations(P_full, P_round, 1)

1 7
2 49
3 343
4 2401
5 16807
6 117649
7 823543
8 5764801
9 40353607
10 282475249
11 1977326743
12 13841287201
13 96889010407
14 678223072849
15 4747561509943
16 33232930569601
17 232630513987207
18 1628413597910449
19 11398895185373143
20 79792266297612001
21 558545864083284007
22 3909821048582988049
23 27368747340080916343
24 36319279229020667897
25 36319279229020667903


In [10]:
# p-degrees mod p
p = 2**31-2**24+1
e = 2
d = 7
t = 8

P_cell = get_polytope_affine_power_permutation(p**e, d)
P_round = compose_permutations(t*P_cell, get_polytope_reduced_linear_permutation(p**e, t), 1)
P_full = P_round
r = 1
pdegrees = []
while True:
    model = MixedIntegerLinearProgram('PPL', maximization=True)
    digits_input_exponents = model.new_variable(integer=True, nonnegative=True)
    model.set_max(digits_input_exponents, t*(p-1))
    rotations_input_exponents = [sum(p**i*digits_input_exponents[(i-j) % e] for i in range(e)) for j in range(e)]
    valuations = model.new_variable(integer=True, nonnegative=True)
    valuation = 0
    for i in range(e):
        valuation += valuations[i]
        constraint_variables_in_polyhedron(model, P_full, [rotations_input_exponents[i], p**i, valuations[i]])
    model.add_constraint(valuation <= 0)
    model.set_objective(sum(digits_input_exponents[i] for i in range(e)))
    res = model.solve()
    print(r, res)
    pdegrees.append(res)
    if res >= t*e*(p-1)-1:
        break
    r += 1
    P_full = compose_permutations(P_full, P_round, 1)

1 7
2 49
3 343
4 2401
5 16807
6 117649
7 823543
8 5764801
9 40353607
10 282475249
11 1977326743
12 13841287201
13 17045651493
14 17045651766
15 17045653676
16 17045667045
17 17045760627
18 17046415707
19 17051001267
20 17083100185
21 17307792611
22 18880639594
23 29890568474
24 34091302911


In [11]:
# properties mod p based on degree F_q
pdiv = divisors(p**e-1)[:-1]
for r, deg in enumerate(qdegrees, start=1):
    cells, subgroup = 0, 0
    cells = floor(deg/(p**e-1))
    deg -= cells*(p**e-1)
    if deg >= pdiv[-1]:
        cells += 1
    else:
        for subgroup in pdiv:
            if subgroup > deg:
                break
    print(r, cells, subgroup, f"{subgroup+1}*q^{cells}")

1 0 8 9*q^0
2 0 64 65*q^0
3 0 381 382*q^0
4 0 3048 3049*q^0
5 0 18112 18113*q^0
6 0 130048 130049*q^0
7 0 862584 862585*q^0
8 0 6242304 6242305*q^0
9 0 49938432 49938433*q^0
10 0 294428672 294428673*q^0
11 0 2130706432 2130706433*q^0
12 0 14132576256 14132576257*q^0
13 0 113060610048 113060610049*q^0
14 0 721599245648 721599245649*q^0
15 0 5222032703488 5222032703489*q^0
16 0 34636763791104 34636763791105*q^0
17 0 250657569767424 250657569767425*q^0
18 0 2005260558139392 2005260558139393*q^0
19 0 11822682040696832 11822682040696833*q^0
20 0 94581456325574656 94581456325574657*q^0
21 0 567488737953447936 567488737953447937*q^0
22 1 0 1*q^1
23 6 141872184488361984 141872184488361985*q^6
24 8 0 1*q^8
25 8 0 1*q^8


In [12]:
# properties mod p as if SHARK-like over F_q
pdiv = divisors(p**e-1)[:-1]
for r, deg in enumerate(qdegrees, start=2):
    cells, subgroup = 0, 0
    cells = floor(deg/(p**e-1))
    deg -= cells*(p**e-1)
    # extend by one round (for one cell)
    if deg > 0:
        model = MixedIntegerLinearProgram('PPL', maximization=True)
        digits_input_exponents = model.new_variable(integer=True, nonnegative=True)
        digits_output_exponents = [floor(deg/(p**j)) % p for j in range(e)]
        model.set_max(digits_input_exponents, p-1)
        rotations_input_exponents = [sum(p**i*digits_input_exponents[(i-j) % e] for i in range(e)) for j in range(e)]
        rotations_output_exponents = [sum(p**i*digits_output_exponents[(i-j) % e] for i in range(e)) for j in range(e)]
        valuations = model.new_variable(integer=True, nonnegative=True)
        valuation = 0
        for i in range(e):
            valuation += valuations[i]
            constraint_variables_in_polyhedron(model, P_cell, [rotations_input_exponents[i], rotations_output_exponents[i], valuations[i]])
        model.add_constraint(valuation <= 0)
        model.set_objective(rotations_input_exponents[0])
        new_deg = model.solve()
        if new_deg >= pdiv[-1]:
            cells += 1
        else:
            for subgroup in pdiv:
                if subgroup > new_deg:
                    break
    print(r, cells, subgroup, f"{subgroup+1}*q^{cells}")

2 0 64 65*q^0
3 0 381 382*q^0
4 0 3048 3049*q^0
5 0 18112 18113*q^0
6 0 130048 130049*q^0
7 0 862584 862585*q^0
8 0 6242304 6242305*q^0
9 0 49938432 49938433*q^0
10 0 294428672 294428673*q^0
11 0 2130706432 2130706433*q^0
12 0 14132576256 14132576257*q^0
13 0 113060610048 113060610049*q^0
14 0 721599245648 721599245649*q^0
15 0 5222032703488 5222032703489*q^0
16 0 34636763791104 34636763791105*q^0
17 0 250657569767424 250657569767425*q^0
18 0 2005260558139392 2005260558139393*q^0
19 0 11822682040696832 11822682040696833*q^0
20 0 94581456325574656 94581456325574657*q^0
21 0 567488737953447936 567488737953447937*q^0
22 1 0 1*q^1
23 1 0 1*q^1
24 6 1134977475906895872 1134977475906895873*q^6
25 8 0 1*q^8
26 8 0 1*q^8


In [13]:
# properties mod p based on degree over F_p
pdiv = divisors(p-1)[:-1]
for r, deg in enumerate(pdegrees, start=1):
    states, subgroup = 0, 0
    states = floor(deg/(p-1))
    deg -= states*(p-1)
    if deg >= pdiv[-1]:
        states += 1
    else:
        for subgroup in pdiv:
            if subgroup > deg:
                break
    print(r, states, subgroup, f"{subgroup+1}*p^{states}")

1 0 8 9*p^0
2 0 64 65*p^0
3 0 508 509*p^0
4 0 4064 4065*p^0
5 0 32512 32513*p^0
6 0 130048 130049*p^0
7 0 1040384 1040385*p^0
8 0 8323072 8323073*p^0
9 0 66584576 66584577*p^0
10 0 532676608 532676609*p^0
11 1 0 1*p^1
12 6 1065353216 1065353217*p^6
13 8 64 65*p^8
14 8 508 509*p^8
15 8 4064 4065*p^8
16 8 16256 16257*p^8
17 8 130048 130049*p^8
18 8 1040384 1040385*p^8
19 8 8323072 8323073*p^8
20 8 66584576 66584577*p^8
21 8 266338304 266338305*p^8
22 9 0 1*p^9
23 14 66584576 66584577*p^14
24 16 0 1*p^16


In [14]:
# properties mod p as if SHARK-like over F_p
pdiv = divisors(p-1)[:-1]
for r, deg in enumerate(pdegrees, start=2):
    cells, states, subgroup = 0, 0, 0
    cells = floor(deg/(e*(p-1)))
    deg -= cells*e*(p-1)
    if deg > 0:
        # # breaks down from here
        # print(deg)
        # model = MixedIntegerLinearProgram('PPL', maximization=True)
        # digits_input_exponents = model.new_variable(integer=True, nonnegative=True)
        # digits_output_exponents = model.new_variable(integer=True, nonnegative=True)
        # model.set_max(digits_input_exponents, p-1)
        # model.set_max(digits_output_exponents, p-1)
        # rotations_input_exponents = [sum(p**i*digits_input_exponents[(i-j) % e] for i in range(e)) for j in range(e)]
        # rotations_output_exponents = [sum(p**i*digits_output_exponents[(i-j) % e] for i in range(e)) for j in range(e)]
        # valuations = model.new_variable(integer=True, nonnegative=True)
        # valuation = 0
        # for i in range(e):
        #     valuation += valuations[i]
        #     constraint_variables_in_polyhedron(model, P_cell, [rotations_input_exponents[i], rotations_output_exponents[i], valuations[i]])
        # model.add_constraint(sum(digits_output_exponents[i] for i in range(e)) <= deg)
        # model.add_constraint(valuation <= 0)
        # model.set_objective(sum(digits_input_exponents[i] for i in range(e)))
        # new_deg = model.solve()
        # print(new_deg)
        # # end of breakdown
        # quick solution, gives an okay upperbound on the degree, only works if the degree of the function is less than p-1
        new_deg = min(e*(p-1), d*deg)
        # end of quick solution
        if new_deg >= e*(p-1):
            cells += 1
        else:
            states += floor(new_deg/(p-1))
            new_deg -= states * (p-1)
            if new_deg >= pdiv[-1]:
                states += 1
                if states == e:
                    states = 0
                    cells += 1
            else:
                for subgroup in pdiv:
                    if subgroup > new_deg:
                        break
    print(r, cells, states, subgroup, f"{subgroup+1}*p^{e*cells+states}")

2 0 0 64 65*p^0
3 0 0 508 509*p^0
4 0 0 4064 4065*p^0
5 0 0 32512 32513*p^0
6 0 0 130048 130049*p^0
7 0 0 1040384 1040385*p^0
8 0 0 8323072 8323073*p^0
9 0 0 66584576 66584577*p^0
10 0 0 532676608 532676609*p^0
11 0 1 0 1*p^1
12 1 0 0 1*p^2
13 4 0 0 1*p^8
14 4 0 508 509*p^8
15 4 0 4064 4065*p^8
16 4 0 16256 16257*p^8
17 4 0 130048 130049*p^8
18 4 0 1040384 1040385*p^8
19 4 0 8323072 8323073*p^8
20 4 0 66584576 66584577*p^8
21 4 0 266338304 266338305*p^8
22 4 1 0 1*p^9
23 5 0 0 1*p^10
24 7 0 532676608 532676609*p^14
25 8 0 0 1*p^16


In [15]:
# q-degrees mod p^2
p = 2**31-2**24+1
e = 2
d = 7
t = 8

P_cell = get_polytope_affine_power_permutation(p**e, d)
P_round = compose_permutations(t*P_cell, get_polytope_reduced_linear_permutation(p**e, t), 1)
P_full = P_round
r = 1
qdegrees2 = []
while True:
    model = MixedIntegerLinearProgram('PPL', maximization=True)
    digits_input_exponents = model.new_variable(integer=True, nonnegative=True)
    model.set_max(digits_input_exponents, t*(p-1))
    rotations_input_exponents = [sum(p**i*digits_input_exponents[(i-j) % e] for i in range(e)) for j in range(e)]
    valuations = model.new_variable(integer=True, nonnegative=True)
    valuation = 0
    for i in range(e):
        valuation += valuations[i]
        constraint_variables_in_polyhedron(model, P_full, [rotations_input_exponents[i], p**i, valuations[i]])
    model.add_constraint(valuation <= 1)
    model.set_objective(rotations_input_exponents[0])
    res = model.solve()
    print(r, res)
    qdegrees2.append(res)
    if res >= t*(p**e-1)-1:
        break
    r += 1
    P_full = compose_permutations(P_full, P_round, 1)

1 31779369325393084423
2 36319279211975016489
3 36319279211975016783
4 36319279211975018841
5 36319279211975033247
6 36319279211975134089
7 36319279211975839983
8 36319279211980781241
9 36319279212015370047
10 36319279212257491689
11 36319279213952343183
12 36319279225816303641
13 36319279229020667903


In [16]:
# properties mod p^2 based on q-degree
pdiv = divisors(p**e-1)[:-1]
for r, deg in enumerate(qdegrees2, start=1):
    cells, subgroup = 0, 0
    cells = floor(deg/(p**e-1))
    deg -= cells*(p**e-1)
    if deg >= pdiv[-1]:
        cells += 1
    else:
        for subgroup in pdiv:
            if subgroup > deg:
                break
    print(r, cells, subgroup, f"{subgroup+1}*q^{cells}")

1 7 8 9*q^7
2 8 0 1*q^8
3 8 0 1*q^8
4 8 0 1*q^8
5 8 0 1*q^8
6 8 0 1*q^8
7 8 0 1*q^8
8 8 0 1*q^8
9 8 0 1*q^8
10 8 0 1*q^8
11 8 0 1*q^8
12 8 0 1*q^8
13 8 0 1*q^8


In [17]:
# p-degrees mod p^2
p = 2**31-2**24+1
e = 2
d = 7
t = 8

P_cell = get_polytope_affine_power_permutation(p**e, d)
P_round = compose_permutations(t*P_cell, get_polytope_reduced_linear_permutation(p**e, t), 1)
P_full = P_round
r = 1
pdegrees2 = []
while True:
    model = MixedIntegerLinearProgram('PPL', maximization=True)
    digits_input_exponents = model.new_variable(integer=True, nonnegative=True)
    model.set_max(digits_input_exponents, t*(p-1))
    rotations_input_exponents = [sum(p**i*digits_input_exponents[(i-j) % e] for i in range(e)) for j in range(e)]
    valuations = model.new_variable(integer=True, nonnegative=True)
    valuation = 0
    for i in range(e):
        valuation += valuations[i]
        constraint_variables_in_polyhedron(model, P_full, [rotations_input_exponents[i], p**i, valuations[i]])
    model.add_constraint(valuation <= 1)
    model.set_objective(sum(digits_input_exponents[i] for i in range(e)))
    res = model.solve()
    print(r, res)
    pdegrees2.append(res)
    if res >= t*e*(p-1)-1:
        break
    r += 1
    P_full = compose_permutations(P_full, P_round, 1)

1 14914945031
2 17045651497
3 17045651791
4 17045653849
5 17045668255
6 17045769097
7 17046474991
8 17051416249
9 17086005055
10 17328126697
11 19022978191
12 30886938649
13 34091302911


In [18]:
# properties mod p^2 based on p-degree
pdiv = divisors(p-1)[:-1]
for r, deg in enumerate(pdegrees2, start=1):
    states, subgroup = 0, 0
    states = floor(deg/(p-1))
    deg -= states*(p-1)
    if deg >= pdiv[-1]:
        states += 1
    else:
        for subgroup in pdiv:
            if subgroup > deg:
                break
    print(r, states, subgroup, f"{subgroup+1}*p^{states}")

1 7 8 9*p^7
2 8 64 65*p^8
3 8 508 509*p^8
4 8 4064 4065*p^8
5 8 32512 32513*p^8
6 8 130048 130049*p^8
7 8 1040384 1040385*p^8
8 8 8323072 8323073*p^8
9 8 66584576 66584577*p^8
10 8 532676608 532676609*p^8
11 9 0 1*p^9
12 14 1065353216 1065353217*p^14
13 16 0 1*p^16


In [19]:
# SHARK-like properties with higher divisibility based on saturating sboxes
p = 2**31-2**24+1
e = 2
d = 7
t = 8

P_cell = get_polytope_affine_power_permutation(p**e, d)
P_round = compose_permutations(t*P_cell, get_polytope_reduced_linear_permutation(p**e, t), 1)
P_full = get_polytope_reduced_linear_permutation(p**e, t)
r = 1
degrees_p2 = []
while True:
    for cells in range(1, t):
        res = 0
        for i in range(e):
            Pred = (P_full & Polyhedron(eqns = [[-p**i, 0, 1, 0], [-(p**e-1)*cells, 1, 0, (p**e-1)]])).linear_transformation(Matrix(ZZ, 1, 3, [[0, 0, 1]]))
            res += min(ceil(x[0]) for x in Pred.vertices_list())
        if res >= 2:
            print(r, cells, res)
    if cells == t-1 and res <= 1:
        break
    r += 1
    P_full = compose_permutations(P_full, P_round, 1)

1 1 2
1 2 2
1 3 4
1 4 4
1 5 6
1 6 6
1 7 8
2 1 2
2 2 2
2 3 2
2 4 2
2 5 2
2 6 2
2 7 2
3 1 2
3 2 2
3 3 2
3 4 2
3 5 2
3 6 2
3 7 2
4 1 2
4 2 2
4 3 2
4 4 2
4 5 2
4 6 2
4 7 2
5 1 2
5 2 2
5 3 2
5 4 2
5 5 2
5 6 2
5 7 2
6 1 2
6 2 2
6 3 2
6 4 2
6 5 2
6 6 2
6 7 2
7 1 2
7 2 2
7 3 2
7 4 2
7 5 2
7 6 2
7 7 2
8 1 2
8 2 2
8 3 2
8 4 2
8 5 2
8 6 2
8 7 2
9 1 2
9 2 2
9 3 2
9 4 2
9 5 2
9 6 2
9 7 2
10 1 2
10 2 2
10 3 2
10 4 2
10 5 2
10 6 2
10 7 2
11 1 2
11 2 2
11 3 2
11 4 2
11 5 2
11 6 2
11 7 2
12 1 2
12 2 2
12 3 2
12 4 2
12 5 2
12 6 2
12 7 2
13 7 2


### degree 4 extention field over 17-bit prime

In [20]:
# q-degrees mod p
p = 2**17-1
e = 4
d = 7
t = 8

P_cell = get_polytope_affine_power_permutation(p**e, d)
P_round = compose_permutations(t*P_cell, get_polytope_reduced_linear_permutation(p**e, t), 1)
P_full = P_round
r = 1
qdegrees = []
while True:
    model = MixedIntegerLinearProgram('PPL', maximization=True)
    digits_input_exponents = model.new_variable(integer=True, nonnegative=True)
    model.set_max(digits_input_exponents, t*(p-1))
    rotations_input_exponents = [sum(p**i*digits_input_exponents[(i-j) % e] for i in range(e)) for j in range(e)]
    valuations = model.new_variable(integer=True, nonnegative=True)
    valuation = 0
    for i in range(e):
        valuation += valuations[i]
        constraint_variables_in_polyhedron(model, P_full, [rotations_input_exponents[i], p**i, valuations[i]])
    model.add_constraint(valuation <= 0)
    model.set_objective(rotations_input_exponents[0])
    res = model.solve()
    print(r, res)
    qdegrees.append(res)
    if res >= t*(p**e-1)-1:
        break
    r += 1
    P_full = compose_permutations(P_full, P_round, 1)

1 7
2 49
3 343
4 2401
5 16807
6 117649
7 823543
8 5764801
9 40353607
10 282475249
11 1977326743
12 13841287201
13 96889010407
14 678223072849
15 4747561509943
16 33232930569601
17 232630513987207
18 1628413597910449
19 11398895185373143
20 79792266297612001
21 558545864083284007
22 3909821048582988049
23 27368747340080916343
24 191581231380566414401
25 1341068619663964900807
26 2361111184665414205427
27 2361111184665414205439


In [21]:
# p^2-degrees mod p
p = 2**17-1
e = 4
d = 7
t = 8

P_cell = get_polytope_affine_power_permutation(p**e, d)
P_round = compose_permutations(t*P_cell, get_polytope_reduced_linear_permutation(p**e, t), 1)
P_full = P_round
r = 1
p2degrees = []
while True:
    model = MixedIntegerLinearProgram('PPL', maximization=True)
    digits_input_exponents = model.new_variable(integer=True, nonnegative=True)
    model.set_max(digits_input_exponents, t*(p-1))
    rotations_input_exponents = [sum(p**i*digits_input_exponents[(i-j) % e] for i in range(e)) for j in range(e)]
    valuations = model.new_variable(integer=True, nonnegative=True)
    valuation = 0
    for i in range(e):
        valuation += valuations[i]
        constraint_variables_in_polyhedron(model, P_full, [rotations_input_exponents[i], p**i, valuations[i]])
    model.add_constraint(valuation <= 0)
    model.set_objective(sum(sum(p**j*digits_input_exponents[2*i+j] for j in range(2)) for i in range(2)))
    res = model.solve()
    print(r, res)
    p2degrees.append(res)
    if res >= t*2*(p**2-1)-1:
        break
    r += 1
    P_full = compose_permutations(P_full, P_round, 1)

1 7
2 49
3 343
4 2401
5 16807
6 117649
7 823543
8 5764801
9 40353607
10 282475249
11 1977326743
12 13841287201
13 96889010407
14 137436856351
15 137436856588
16 137436858246
17 137436869853
18 137436951099
19 137437519825
20 137441500904
21 137469368456
22 137664441322
23 139029951385
24 148588521825
25 215498514908
26 274873712639


In [22]:
# p-degrees mod p
p = 2**17-1
e = 4
d = 7
t = 8
P_cell = get_polytope_affine_power_permutation(p**e, d)
P_round = compose_permutations(t*P_cell, get_polytope_reduced_linear_permutation(p**e, t), 1)
P_full = P_round
r = 1
pdegrees = []
while True:
    model = MixedIntegerLinearProgram('PPL', maximization=True)
    digits_input_exponents = model.new_variable(integer=True, nonnegative=True)
    model.set_max(digits_input_exponents, t*(p-1))
    rotations_input_exponents = [sum(p**i*digits_input_exponents[(i-j) % e] for i in range(e)) for j in range(e)]
    valuations = model.new_variable(integer=True, nonnegative=True)
    valuation = 0
    for i in range(e):
        valuation += valuations[i]
        constraint_variables_in_polyhedron(model, P_full, [rotations_input_exponents[i], p**i, valuations[i]])
    model.add_constraint(valuation <= 0)
    model.set_objective(sum(digits_input_exponents[i] for i in range(e)))
    res = model.solve()
    print(r, res)
    pdegrees.append(res)
    if res >= t*e*(p-1)-1:
        break
    r += 1
    P_full = compose_permutations(P_full, P_round, 1)

1 7
2 49
3 343
4 2401
5 16807
6 117649
7 823543
8 1048595
9 1048859
10 1050707
11 1063637
12 1154153
13 1787762
14 2097151
15 2097388
16 2099046
17 2110653
18 2191899
19 2760625
20 3145707
21 3145920
22 3147408
23 3157826
24 3230753
25 3741239
26 4194239


In [23]:
# properties mod p based on degree F_q
pdiv = divisors(p**e-1)[:-1]
for r, deg in enumerate(qdegrees, start=1):
    cells, subgroup = 0, 0
    cells = floor(deg/(p**e-1))
    deg -= cells*(p**e-1)
    if deg >= pdiv[-1]:
        cells += 1
    else:
        for subgroup in pdiv:
            if subgroup > deg:
                break
    print(r, cells, subgroup, f"{subgroup+1}*q^{cells}")

1 0 8 9*q^0
2 0 51 52*q^0
3 0 384 385*q^0
4 0 2496 2497*q^0
5 0 16904 16905*q^0
6 0 122880 122881*q^0
7 0 824070 824071*q^0
8 0 6254180 6254181*q^0
9 0 40652170 40652171*q^0
10 0 286326784 286326785*q^0
11 0 1982262351 1982262352*q^0
12 0 13958430720 13958430721*q^0
13 0 99906772992 99906772993*q^0
14 0 679255232276 679255232277*q^0
15 0 4840067956736 4840067956737*q^0
16 0 33576360861696 33576360861697*q^0
17 0 235953312890880 235953312890881*q^0
18 0 1643266602172416 1643266602172417*q^0
19 0 11639805098721280 11639805098721281*q^0
20 0 83466882942074880 83466882942074881*q^0
21 0 567574804006109184 567574804006109185*q^0
22 0 4340277912987893760 4340277912987893761*q^0
23 0 29513889808317677568 29513889808317677569*q^0
24 1 0 1*q^1
25 5 0 1*q^5
26 8 0 1*q^8
27 8 0 1*q^8


In [24]:
# properties mod p as if SHARK-like over F_q
pdiv = divisors(p**e-1)[:-1]
for r, deg in enumerate(qdegrees, start=2):
    cells, subgroup = 0, 0
    cells = floor(deg/(p**e-1))
    deg -= cells*(p**e-1)
    # extend by one round (for one cell)
    if deg > 0:
        model = MixedIntegerLinearProgram('PPL', maximization=True)
        digits_input_exponents = model.new_variable(integer=True, nonnegative=True)
        digits_output_exponents = [floor(deg/(p**j)) % p for j in range(e)]
        model.set_max(digits_input_exponents, p-1)
        rotations_input_exponents = [sum(p**i*digits_input_exponents[(i-j) % e] for i in range(e)) for j in range(e)]
        rotations_output_exponents = [sum(p**i*digits_output_exponents[(i-j) % e] for i in range(e)) for j in range(e)]
        valuations = model.new_variable(integer=True, nonnegative=True)
        valuation = 0
        for i in range(e):
            valuation += valuations[i]
            constraint_variables_in_polyhedron(model, P_cell, [rotations_input_exponents[i], rotations_output_exponents[i], valuations[i]])
        model.add_constraint(valuation <= 0)
        model.set_objective(rotations_input_exponents[0])
        new_deg = model.solve()
        if new_deg >= pdiv[-1]:
            cells += 1
        else:
            for subgroup in pdiv:
                if subgroup > new_deg:
                    break
    print(r, cells, subgroup, f"{subgroup+1}*q^{cells}")

2 0 51 52*q^0
3 0 384 385*q^0
4 0 2496 2497*q^0
5 0 16904 16905*q^0
6 0 122880 122881*q^0
7 0 824070 824071*q^0
8 0 6254180 6254181*q^0
9 0 40652170 40652171*q^0
10 0 286326784 286326785*q^0
11 0 1982262351 1982262352*q^0
12 0 13958430720 13958430721*q^0
13 0 99906772992 99906772993*q^0
14 0 679255232276 679255232277*q^0
15 0 4840067956736 4840067956737*q^0
16 0 33576360861696 33576360861697*q^0
17 0 235953312890880 235953312890881*q^0
18 0 1643266602172416 1643266602172417*q^0
19 0 11639805098721280 11639805098721281*q^0
20 0 83466882942074880 83466882942074881*q^0
21 0 567574804006109184 567574804006109185*q^0
22 0 4340277912987893760 4340277912987893761*q^0
23 0 29513889808317677568 29513889808317677569*q^0
24 1 0 1*q^1
25 1 0 1*q^1
26 5 0 1*q^5
27 8 0 1*q^8
28 8 0 1*q^8


In [25]:
# properties mod p based on degree over F_{p^2}
pdiv = divisors(p**2-1)[:-1]
for r, deg in enumerate(p2degrees, start=1):
    states, subgroup = 0, 0
    states = floor(deg/(p**2-1))
    deg -= states*(p**2-1)
    if deg >= pdiv[-1]:
        states += 1
    else:
        for subgroup in pdiv:
            if subgroup > deg:
                break
    print(r, states, subgroup, f"{subgroup+1}*p^{{2*{states}}}")

1 0 8 9*p^{2*0}
2 0 51 52*p^{2*0}
3 0 384 385*p^{2*0}
4 0 2560 2561*p^{2*0}
5 0 17408 17409*p^{2*0}
6 0 122880 122881*p^{2*0}
7 0 835584 835585*p^{2*0}
8 0 6316032 6316033*p^{2*0}
9 0 42106880 42106881*p^{2*0}
10 0 286326784 286326785*p^{2*0}
11 0 2147450880 2147450881*p^{2*0}
12 1 0 1*p^{2*1}
13 6 0 1*p^{2*6}
14 8 32 33*p^{2*8}
15 8 272 273*p^{2*8}
16 8 2040 2041*p^{2*8}
17 8 15360 15361*p^{2*8}
18 8 98304 98305*p^{2*8}
19 8 696320 696321*p^{2*8}
20 8 5263360 5263361*p^{2*8}
21 8 33423360 33423361*p^{2*8}
22 8 252641280 252641281*p^{2*8}
23 8 1717960704 1717960705*p^{2*8}
24 9 0 1*p^{2*9}
25 13 0 1*p^{2*13}
26 16 0 1*p^{2*16}


In [26]:
# properties mod p as if SHARK-like over F_{p^2}
pdiv = divisors(p**2-1)[:-1]
for r, deg in enumerate(p2degrees, start=2):
    cells, states, subgroup = 0, 0, 0
    cells = floor(deg/(2*(p**2-1)))
    deg -= cells*2*(p**2-1)
    if deg > 0:
        # # breaks down from here
        # print(deg)
        # model = MixedIntegerLinearProgram('PPL', maximization=True)
        # digits_input_exponents = model.new_variable(integer=True, nonnegative=True)
        # digits_output_exponents = model.new_variable(integer=True, nonnegative=True)
        # model.set_max(digits_input_exponents, p-1)
        # model.set_max(digits_output_exponents, p-1)
        # rotations_input_exponents = [sum(p**i*digits_input_exponents[(i-j) % e] for i in range(e)) for j in range(e)]
        # rotations_output_exponents = [sum(p**i*digits_output_exponents[(i-j) % e] for i in range(e)) for j in range(e)]
        # valuations = model.new_variable(integer=True, nonnegative=True)
        # valuation = 0
        # for i in range(e):
        #     valuation += valuations[i]
        #     constraint_variables_in_polyhedron(model, P_cell, [rotations_input_exponents[i], rotations_output_exponents[i], valuations[i]])
        # model.add_constraint(sum(digits_output_exponents[i] for i in range(e)) <= deg)
        # model.add_constraint(valuation <= 0)
        # model.set_objective(sum(digits_input_exponents[i] for i in range(e)))
        # new_deg = model.solve()
        # print(new_deg)
        # # end of breakdown
        # quick solution, gives an okay upperbound on the degree, only works if the degree of the function is less than p-1
        new_deg = min(2*(p**2-1), d*deg)
        # end of quick solution
        if new_deg >= 2*(p**2-1):
            cells += 1
        else:
            states += floor(new_deg/(p**2-1))
            new_deg -= states * (p**2-1)
            if new_deg >= pdiv[-1]:
                states += 1
                if states == e:
                    states = 0
                    cells += 1
            else:
                for subgroup in pdiv:
                    if subgroup > new_deg:
                        break
    print(r, cells, states, subgroup, f"{subgroup+1}*p^{{2*{2*cells+states}}}")

2 0 0 51 52*p^{2*0}
3 0 0 384 385*p^{2*0}
4 0 0 2560 2561*p^{2*0}
5 0 0 17408 17409*p^{2*0}
6 0 0 122880 122881*p^{2*0}
7 0 0 835584 835585*p^{2*0}
8 0 0 6316032 6316033*p^{2*0}
9 0 0 42106880 42106881*p^{2*0}
10 0 0 286326784 286326785*p^{2*0}
11 0 0 2147450880 2147450881*p^{2*0}
12 0 1 0 1*p^{2*1}
13 1 0 0 1*p^{2*2}
14 3 0 0 1*p^{2*6}
15 4 0 240 241*p^{2*8}
16 4 0 1920 1921*p^{2*8}
17 4 0 15360 15361*p^{2*8}
18 4 0 98304 98305*p^{2*8}
19 4 0 696320 696321*p^{2*8}
20 4 0 5263360 5263361*p^{2*8}
21 4 0 33423360 33423361*p^{2*8}
22 4 0 252641280 252641281*p^{2*8}
23 4 0 1717960704 1717960705*p^{2*8}
24 4 1 0 1*p^{2*9}
25 5 0 0 1*p^{2*10}
26 7 0 0 1*p^{2*14}
27 8 0 0 1*p^{2*16}


In [27]:
# properties mod p based on degree over F_p
pdiv = divisors(p-1)[:-1]
for r, deg in enumerate(pdegrees, start=1):
    states, subgroup = 0, 0
    states = floor(deg/(p-1))
    deg -= states*(p-1)
    if deg >= pdiv[-1]:
        states += 1
    else:
        for subgroup in pdiv:
            if subgroup > deg:
                break
    print(r, states, subgroup, f"{subgroup+1}*p^{states}")

1 0 10 11*p^0
2 0 51 52*p^0
3 0 510 511*p^0
4 0 2570 2571*p^0
5 0 21845 21846*p^0
6 1 0 1*p^1
7 6 43690 43691*p^6
8 8 51 52*p^8
9 8 510 511*p^8
10 8 2570 2571*p^8
11 8 21845 21846*p^8
12 9 0 1*p^9
13 14 0 1*p^14
14 16 34 35*p^16
15 16 510 511*p^16
16 16 2570 2571*p^16
17 16 21845 21846*p^16
18 17 0 1*p^17
19 21 8738 8739*p^21
20 24 30 31*p^24
21 24 255 256*p^24
22 24 2570 2571*p^24
23 24 13107 13108*p^24
24 25 0 1*p^25
25 29 0 1*p^29
26 32 0 1*p^32


In [28]:
# properties mod p as if SHARK-like over F_p
pdiv = divisors(p-1)[:-1]
for r, deg in enumerate(pdegrees, start=2):
    cells, states, subgroup = 0, 0, 0
    cells = floor(deg/(e*(p-1)))
    deg -= cells*e*(p-1)
    if deg > 0:
        # # breaks down from here
        # print(deg)
        # model = MixedIntegerLinearProgram('PPL', maximization=True)
        # digits_input_exponents = model.new_variable(integer=True, nonnegative=True)
        # digits_output_exponents = model.new_variable(integer=True, nonnegative=True)
        # model.set_max(digits_input_exponents, p-1)
        # model.set_max(digits_output_exponents, p-1)
        # rotations_input_exponents = [sum(p**i*digits_input_exponents[(i-j) % e] for i in range(e)) for j in range(e)]
        # rotations_output_exponents = [sum(p**i*digits_output_exponents[(i-j) % e] for i in range(e)) for j in range(e)]
        # valuations = model.new_variable(integer=True, nonnegative=True)
        # valuation = 0
        # for i in range(e):
        #     valuation += valuations[i]
        #     constraint_variables_in_polyhedron(model, P_cell, [rotations_input_exponents[i], rotations_output_exponents[i], valuations[i]])
        # model.add_constraint(sum(digits_output_exponents[i] for i in range(e)) <= deg)
        # model.add_constraint(valuation <= 0)
        # model.set_objective(sum(digits_input_exponents[i] for i in range(e)))
        # new_deg = model.solve()
        # print(new_deg)
        # # end of breakdown
        # quick solution, gives an okay upperbound on the degree, only works if the degree of the function is less than p-1
        new_deg = min(e*(p-1), d*deg)
        # end of quick solution
        if new_deg >= e*(p-1):
            cells += 1
        else:
            states += floor(new_deg/(p-1))
            new_deg -= states * (p-1)
            if new_deg >= pdiv[-1]:
                states += 1
                if states == e:
                    states = 0
                    cells += 1
            else:
                for subgroup in pdiv:
                    if subgroup > new_deg:
                        break
    print(r, cells, states, subgroup, f"{subgroup+1}*p^{e*cells+states}")

2 0 0 51 52*p^0
3 0 0 510 511*p^0
4 0 0 2570 2571*p^0
5 0 0 21845 21846*p^0
6 0 1 0 1*p^1
7 1 0 0 1*p^4
8 2 0 0 1*p^8
9 2 0 255 256*p^8
10 2 0 2570 2571*p^8
11 2 0 21845 21846*p^8
12 2 1 0 1*p^9
13 3 0 0 1*p^12
14 4 0 0 1*p^16
15 4 0 255 256*p^16
16 4 0 2570 2571*p^16
17 4 0 21845 21846*p^16
18 4 1 0 1*p^17
19 5 0 0 1*p^20
20 6 0 0 1*p^24
21 6 0 255 256*p^24
22 6 0 2570 2571*p^24
23 6 0 13107 13108*p^24
24 6 1 0 1*p^25
25 7 0 0 1*p^28
26 8 0 0 1*p^32
27 8 0 0 1*p^32


In [29]:
# q-degrees mod p^2
p = 2**17-1
e = 4
d = 7
t = 8

P_cell = get_polytope_affine_power_permutation(p**e, d)
P_round = compose_permutations(t*P_cell, get_polytope_reduced_linear_permutation(p**e, t), 1)
P_full = P_round
r = 1
qdegrees2 = []
while True:
    model = MixedIntegerLinearProgram('PPL', maximization=True)
    digits_input_exponents = model.new_variable(integer=True, nonnegative=True)
    model.set_max(digits_input_exponents, t*(p-1))
    rotations_input_exponents = [sum(p**i*digits_input_exponents[(i-j) % e] for i in range(e)) for j in range(e)]
    valuations = model.new_variable(integer=True, nonnegative=True)
    valuation = 0
    for i in range(e):
        valuation += valuations[i]
        constraint_variables_in_polyhedron(model, P_full, [rotations_input_exponents[i], p**i, valuations[i]])
    model.add_constraint(valuation <= 1)
    model.set_objective(rotations_input_exponents[0])
    res = model.solve()
    print(r, res)
    qdegrees2.append(res)
    if res >= t*(p**e-1)-1:
        break
    r += 1
    P_full = compose_permutations(P_full, P_round, 1)

1 2065972286582237429767
2 2361093170679218438201
3 2361093170679218438495
4 2361093170679218440553
5 2361093170679218454959
6 2361093170679218555801
7 2361093170679219261695
8 2361093170679224202953
9 2361093170679258791759
10 2361093170679500913401
11 2361093170681195764895
12 2361093170693059725353
13 2361093170776107448559
14 2361093171357441511001
15 2361093175426779948095
16 2361093203912149007753
17 2361093403309732425359
18 2361094799092816348601
19 2361104569574403811295
20 2361111184665414205439


In [30]:
# properties mod p^2 based on q-degree
pdiv = divisors(p**e-1)[:-1]
for r, deg in enumerate(qdegrees2, start=1):
    cells, subgroup = 0, 0
    cells = floor(deg/(p**e-1))
    deg -= cells*(p**e-1)
    if deg >= pdiv[-1]:
        cells += 1
    else:
        for subgroup in pdiv:
            if subgroup > deg:
                break
    print(r, cells, subgroup, f"{subgroup+1}*q^{cells}")

1 7 8 9*q^7
2 8 0 1*q^8
3 8 0 1*q^8
4 8 0 1*q^8
5 8 0 1*q^8
6 8 0 1*q^8
7 8 0 1*q^8
8 8 0 1*q^8
9 8 0 1*q^8
10 8 0 1*q^8
11 8 0 1*q^8
12 8 0 1*q^8
13 8 0 1*q^8
14 8 0 1*q^8
15 8 0 1*q^8
16 8 0 1*q^8
17 8 0 1*q^8
18 8 0 1*q^8
19 8 0 1*q^8
20 8 0 1*q^8


In [31]:
# p^2-degrees mod p^2
p = 2**17-1
e = 4
d = 7
t = 8

P_cell = get_polytope_affine_power_permutation(p**e, d)
P_round = compose_permutations(t*P_cell, get_polytope_reduced_linear_permutation(p**e, t), 1)
P_full = P_round
r = 1
p2degrees2 = []
while True:
    model = MixedIntegerLinearProgram('PPL', maximization=True)
    digits_input_exponents = model.new_variable(integer=True, nonnegative=True)
    model.set_max(digits_input_exponents, t*(p-1))
    rotations_input_exponents = [sum(p**i*digits_input_exponents[(i-j) % e] for i in range(e)) for j in range(e)]
    valuations = model.new_variable(integer=True, nonnegative=True)
    valuation = 0
    for i in range(e):
        valuation += valuations[i]
        constraint_variables_in_polyhedron(model, P_full, [rotations_input_exponents[i], p**i, valuations[i]])
    model.add_constraint(valuation <= 1)
    model.set_objective(sum(sum(p**j*digits_input_exponents[2*i+j] for j in range(2)) for i in range(2)))
    res = model.solve()
    print(r, res)
    p2degrees2.append(res)
    if res >= t*2*(p**2-1)-1:
        break
    r += 1
    P_full = compose_permutations(P_full, P_round, 1)

1 120257249287
2 137435807801
3 137435808095
4 137435810153
5 137435824559
6 137435925401
7 137436631295
8 137441572553
9 137476161359
10 137718283001
11 139413134495
12 151277094953
13 234324818159
14 274872664111
15 274872664348
16 274872666006
17 274872677613
18 274872758859
19 274873327585
20 274873712639


In [32]:
# properties mod p^2 based on p^2-degree
pdiv = divisors(p**2-1)[:-1]
for r, deg in enumerate(p2degrees2, start=1):
    states, subgroup = 0, 0
    states = floor(deg/(p**2-1))
    deg -= states*(p**2-1)
    if deg >= pdiv[-1]:
        states += 1
    else:
        for subgroup in pdiv:
            if subgroup > deg:
                break
    print(r, states, subgroup, f"{subgroup+1}*p^{2*states}")

1 7 8 9*p^14
2 8 0 1*p^16
3 8 0 1*p^16
4 8 0 1*p^16
5 8 0 1*p^16
6 8 0 1*p^16
7 8 0 1*p^16
8 8 5263360 5263361*p^16
9 8 42106880 42106881*p^16
10 8 286326784 286326785*p^16
11 8 2147450880 2147450881*p^16
12 9 0 1*p^18
13 14 0 1*p^28
14 16 0 1*p^32
15 16 0 1*p^32
16 16 0 1*p^32
17 16 0 1*p^32
18 16 0 1*p^32
19 16 0 1*p^32
20 16 0 1*p^32


In [33]:
# p-degrees mod p^2
p = 2**17-1
e = 4
d = 7
t = 8

P_cell = get_polytope_affine_power_permutation(p**e, d)
P_round = compose_permutations(t*P_cell, get_polytope_reduced_linear_permutation(p**e, t), 1)
P_full = P_round
r = 1
pdegrees2 = []
while True:
    model = MixedIntegerLinearProgram('PPL', maximization=True)
    digits_input_exponents = model.new_variable(integer=True, nonnegative=True)
    model.set_max(digits_input_exponents, t*(p-1))
    rotations_input_exponents = [sum(p**i*digits_input_exponents[(i-j) % e] for i in range(e)) for j in range(e)]
    valuations = model.new_variable(integer=True, nonnegative=True)
    valuation = 0
    for i in range(e):
        valuation += valuations[i]
        constraint_variables_in_polyhedron(model, P_full, [rotations_input_exponents[i], p**i, valuations[i]])
    model.add_constraint(valuation <= 1)
    model.set_objective(sum(digits_input_exponents[i] for i in range(e)))
    res = model.solve()
    print(r, res)
    pdegrees2.append(res)
    if res >= t*e*(p-1)-1:
        break
    r += 1
    P_full = compose_permutations(P_full, P_round, 1)

1 917497
2 1048601
3 1048895
4 1050953
5 1065359
6 1166201
7 1872095
8 2097155
9 2097419
10 2099267
11 2112197
12 2202713
13 2836322
14 3145711
15 3145948
16 3147606
17 3159213
18 3240459
19 3809185
20 4194239


In [34]:
# properties mod p^2 based on p-degree
pdiv = divisors(p-1)[:-1]
for r, deg in enumerate(pdegrees2, start=1):
    states, subgroup = 0, 0
    states = floor(deg/(p-1))
    deg -= states*(p-1)
    if deg >= pdiv[-1]:
        states += 1
    else:
        for subgroup in pdiv:
            if subgroup > deg:
                break
    print(r, states, subgroup, f"{subgroup+1}*p^{states}")

1 7 10 11*p^7
2 8 51 52*p^8
3 8 510 511*p^8
4 8 2570 2571*p^8
5 8 21845 21846*p^8
6 9 0 1*p^9
7 14 43690 43691*p^14
8 16 51 52*p^16
9 16 510 511*p^16
10 16 2570 2571*p^16
11 16 21845 21846*p^16
12 17 0 1*p^17
13 22 0 1*p^22
14 24 34 35*p^24
15 24 510 511*p^24
16 24 2570 2571*p^24
17 24 21845 21846*p^24
18 25 0 1*p^25
19 29 8738 8739*p^29
20 32 0 1*p^32


In [35]:
# SHARK-like properties with higher divisibility based on saturating sboxes
p = 2**17-1
e = 4
d = 7
t = 8

P_cell = get_polytope_affine_power_permutation(p**e, d)
P_round = compose_permutations(t*P_cell, get_polytope_reduced_linear_permutation(p**e, t), 1)
P_full = get_polytope_reduced_linear_permutation(p**e, t)
r = 1
degrees_p2 = []
while True:
    for cells in range(1, t):
        res = 0
        for i in range(e):
            Pred = (P_full & Polyhedron(eqns = [[-p**i, 0, 1, 0], [-(p**e-1)*cells, 1, 0, (p**e-1)]])).linear_transformation(Matrix(ZZ, 1, 3, [[0, 0, 1]]))
            res += min(ceil(x[0]) for x in Pred.vertices_list())
        if res >= 2:
            print(r, cells, res)
    if cells == t-1 and res <= 1:
        break
    r += 1
    P_full = compose_permutations(P_full, P_round, 1)

1 1 4
1 2 4
1 3 8
1 4 8
1 5 12
1 6 12
1 7 16
2 1 4
2 2 4
2 3 4
2 4 4
2 5 4
2 6 4
2 7 4
3 1 4
3 2 4
3 3 4
3 4 4
3 5 4
3 6 4
3 7 4
4 1 4
4 2 4
4 3 4
4 4 4
4 5 4
4 6 4
4 7 4
5 1 4
5 2 4
5 3 4
5 4 4
5 5 4
5 6 4
5 7 4
6 1 4
6 2 4
6 3 4
6 4 4
6 5 4
6 6 4
6 7 4
7 1 4
7 2 4
7 3 4
7 4 4
7 5 4
7 6 4
7 7 4
8 1 3
8 2 3
8 3 3
8 4 3
8 5 3
8 6 3
8 7 4
9 1 3
9 2 3
9 3 3
9 4 3
9 5 3
9 6 3
9 7 3
10 1 3
10 2 3
10 3 3
10 4 3
10 5 3
10 6 3
10 7 3
11 1 3
11 2 3
11 3 3
11 4 3
11 5 3
11 6 3
11 7 3
12 1 3
12 2 3
12 3 3
12 4 3
12 5 3
12 6 3
12 7 3
13 1 3
13 2 3
13 3 3
13 4 3
13 5 3
13 6 3
13 7 3
14 1 2
14 2 2
14 3 2
14 4 2
14 5 2
14 6 3
14 7 3
15 1 2
15 2 2
15 3 2
15 4 2
15 5 2
15 6 2
15 7 2
16 1 2
16 2 2
16 3 2
16 4 2
16 5 2
16 6 2
16 7 2
17 1 2
17 2 2
17 3 2
17 4 2
17 5 2
17 6 2
17 7 2
18 1 2
18 2 2
18 3 2
18 4 2
18 5 2
18 6 2
18 7 2
19 1 2
19 2 2
19 3 2
19 4 2
19 5 2
19 6 2
19 7 2
20 6 2
20 7 2


### degree 8 extention field over 8-bit prime

In [36]:
# q-degrees mod p
p = 2**8+1
e = 8
d = 7
t = 8

P_cell = get_polytope_affine_power_permutation(p**e, d)
P_round = compose_permutations(t*P_cell, get_polytope_reduced_linear_permutation(p**e, t), 1)
P_full = P_round
r = 1
qdegrees = []
while True:
    model = MixedIntegerLinearProgram('PPL', maximization=True)
    digits_input_exponents = model.new_variable(integer=True, nonnegative=True)
    model.set_max(digits_input_exponents, t*(p-1))
    rotations_input_exponents = [sum(p**i*digits_input_exponents[(i-j) % e] for i in range(e)) for j in range(e)]
    valuations = model.new_variable(integer=True, nonnegative=True)
    valuation = 0
    for i in range(e):
        valuation += valuations[i]
        constraint_variables_in_polyhedron(model, P_full, [rotations_input_exponents[i], p**i, valuations[i]])
    model.add_constraint(valuation <= 0)
    model.set_objective(rotations_input_exponents[0])
    res = model.solve()
    print(r, res)
    qdegrees.append(res)
    if res >= t*(p**e-1)-1:
        break
    r += 1
    P_full = compose_permutations(P_full, P_round, 1)

1 7
2 49
3 343
4 2401
5 16807
6 117649
7 823543
8 5764801
9 40353607
10 282475249
11 1977326743
12 13841287201
13 96889010407
14 678223072849
15 4747561509943
16 33232930569601
17 232630513987207
18 1628413597910449
19 11398895185373143
20 79792266297612001
21 558545864083284007
22 3909821048582988049
23 27368747340080916343
24 152249183996808806377
25 152249183996808806399


In [37]:
p = 2**8+1
e = 8
d = 7
t = 8
P_cell = get_polytope_affine_power_permutation(p**e, d)
P_round = compose_permutations(t*P_cell, get_polytope_reduced_linear_permutation(p**e, t), 1)
P_full = P_round
r = 1
pdegrees = []
while True:
    model = MixedIntegerLinearProgram('PPL', maximization=True)
    digits_input_exponents = model.new_variable(integer=True, nonnegative=True)
    model.set_max(digits_input_exponents, t*(p-1))
    rotations_input_exponents = [sum(p**i*digits_input_exponents[(i-j) % e] for i in range(e)) for j in range(e)]
    valuations = model.new_variable(integer=True, nonnegative=True)
    valuation = 0
    for i in range(e):
        valuation += valuations[i]
        constraint_variables_in_polyhedron(model, P_full, [rotations_input_exponents[i], p**i, valuations[i]])
    model.add_constraint(valuation <= 0)
    model.set_objective(sum(digits_input_exponents[i] for i in range(e)))
    res = model.solve()
    print(r, res)
    pdegrees.append(res)
    if res >= t*e*(p-1)-1:
        break
    r += 1
    P_full = compose_permutations(P_full, P_round, 1)

1 7
2 49
3 343
4 2049
5 2105
6 2497
7 4100
8 4175
9 4698
10 6152
11 6252
12 6951
13 8206
14 8339
15 9272
16 10261
17 10439
18 11684
19 12319
20 12556
21 14218
22 14380
23 14697
24 16383


In [38]:
# properties mod p based on degree F_q
pdiv = divisors(p**e-1)[:-1]
for r, deg in enumerate(qdegrees, start=1):
    cells, subgroup = 0, 0
    cells = floor(deg/(p**e-1))
    deg -= cells*(p**e-1)
    if deg >= pdiv[-1]:
        cells += 1
    else:
        for subgroup in pdiv:
            if subgroup > deg:
                break
    print(r, cells, subgroup, f"{subgroup+1}*q^{cells}")

1 0 8 9*q^0
2 0 50 51*q^0
3 0 344 345*q^0
4 0 2550 2551*q^0
5 0 17200 17201*q^0
6 0 126816 126817*q^0
7 0 825600 825601*q^0
8 0 5793906 5793907*q^0
9 0 40581120 40581121*q^0
10 0 287449600 287449601*q^0
11 0 1977653248 1977653249*q^0
12 0 14832399360 14832399361*q^0
13 0 98540507904 98540507905*q^0
14 0 697995264320 697995264321*q^0
15 0 4927025395200 4927025395201*q^0
16 0 33503772687360 33503772687361*q^0
17 0 233224852936288 233224852936289*q^0
18 0 1728847020312600 1728847020312601*q^0
19 0 11661242646814400 11661242646814401*q^0
20 0 88516967440005120 88516967440005121*q^0
21 0 559739647047091200 559739647047091201*q^0
22 0 4757786999900275200 4757786999900275201*q^0
23 1 9515573999800550400 9515573999800550401*q^1
24 8 0 1*q^8
25 8 0 1*q^8


In [39]:
# properties mod p as if SHARK-like over F_q
pdiv = divisors(p**e-1)[:-1]
for r, deg in enumerate(qdegrees, start=2):
    cells, subgroup = 0, 0
    cells = floor(deg/(p**e-1))
    deg -= cells*(p**e-1)
    # extend by one round (for one cell)
    if deg > 0:
        model = MixedIntegerLinearProgram('PPL', maximization=True)
        digits_input_exponents = model.new_variable(integer=True, nonnegative=True)
        digits_output_exponents = [floor(deg/(p**j)) % p for j in range(e)]
        model.set_max(digits_input_exponents, p-1)
        rotations_input_exponents = [sum(p**i*digits_input_exponents[(i-j) % e] for i in range(e)) for j in range(e)]
        rotations_output_exponents = [sum(p**i*digits_output_exponents[(i-j) % e] for i in range(e)) for j in range(e)]
        valuations = model.new_variable(integer=True, nonnegative=True)
        valuation = 0
        for i in range(e):
            valuation += valuations[i]
            constraint_variables_in_polyhedron(model, P_cell, [rotations_input_exponents[i], rotations_output_exponents[i], valuations[i]])
        model.add_constraint(valuation <= 0)
        model.set_objective(rotations_input_exponents[0])
        new_deg = model.solve()
        if new_deg >= pdiv[-1]:
            cells += 1
        else:
            for subgroup in pdiv:
                if subgroup > new_deg:
                    break
    print(r, cells, subgroup, f"{subgroup+1}*q^{cells}")

2 0 50 51*q^0
3 0 344 345*q^0
4 0 2550 2551*q^0
5 0 17200 17201*q^0
6 0 126816 126817*q^0
7 0 825600 825601*q^0
8 0 5793906 5793907*q^0
9 0 40581120 40581121*q^0
10 0 287449600 287449601*q^0
11 0 1977653248 1977653249*q^0
12 0 14832399360 14832399361*q^0
13 0 98540507904 98540507905*q^0
14 0 697995264320 697995264321*q^0
15 0 4927025395200 4927025395201*q^0
16 0 33503772687360 33503772687361*q^0
17 0 233224852936288 233224852936289*q^0
18 0 1728847020312600 1728847020312601*q^0
19 0 11661242646814400 11661242646814401*q^0
20 0 88516967440005120 88516967440005121*q^0
21 0 559739647047091200 559739647047091201*q^0
22 0 4757786999900275200 4757786999900275201*q^0
23 1 0 1*q^1
24 2 0 1*q^2
25 8 0 1*q^8
26 8 0 1*q^8


In [40]:
# properties mod p based on degree over F_p
pdiv = divisors(p-1)[:-1]
for r, deg in enumerate(pdegrees, start=1):
    states, subgroup = 0, 0
    states = floor(deg/(p-1))
    deg -= states*(p-1)
    if deg >= pdiv[-1]:
        states += 1
    else:
        for subgroup in pdiv:
            if subgroup > deg:
                break
    print(r, states, subgroup, f"{subgroup+1}*p^{states}")

1 0 8 9*p^0
2 0 64 65*p^0
3 1 128 129*p^1
4 8 2 3*p^8
5 8 64 65*p^8
6 10 0 1*p^10
7 16 8 9*p^16
8 16 128 129*p^16
9 18 128 129*p^18
10 24 16 17*p^24
11 24 128 129*p^24
12 27 64 65*p^27
13 32 16 17*p^32
14 33 0 1*p^33
15 36 64 65*p^36
16 40 32 33*p^40
17 41 0 1*p^41
18 46 0 1*p^46
19 48 32 33*p^48
20 49 16 17*p^49
21 56 0 1*p^56
22 56 64 65*p^56
23 57 128 129*p^57
24 64 0 1*p^64


In [41]:
# properties mod p as if SHARK-like over F_p
pdiv = divisors(p-1)[:-1]
for r, deg in enumerate(pdegrees, start=2):
    cells, states, subgroup = 0, 0, 0
    cells = floor(deg/(e*(p-1)))
    deg -= cells*e*(p-1)
    if deg > 0:
        # # breaks down from here
        # print(deg)
        # model = MixedIntegerLinearProgram('PPL', maximization=True)
        # digits_input_exponents = model.new_variable(integer=True, nonnegative=True)
        # digits_output_exponents = model.new_variable(integer=True, nonnegative=True)
        # model.set_max(digits_input_exponents, p-1)
        # model.set_max(digits_output_exponents, p-1)
        # rotations_input_exponents = [sum(p**i*digits_input_exponents[(i-j) % e] for i in range(e)) for j in range(e)]
        # rotations_output_exponents = [sum(p**i*digits_output_exponents[(i-j) % e] for i in range(e)) for j in range(e)]
        # valuations = model.new_variable(integer=True, nonnegative=True)
        # valuation = 0
        # for i in range(e):
        #     valuation += valuations[i]
        #     constraint_variables_in_polyhedron(model, P_cell, [rotations_input_exponents[i], rotations_output_exponents[i], valuations[i]])
        # model.add_constraint(sum(digits_output_exponents[i] for i in range(e)) <= deg)
        # model.add_constraint(valuation <= 0)
        # model.set_objective(sum(digits_input_exponents[i] for i in range(e)))
        # new_deg = model.solve()
        # print(new_deg)
        # # end of breakdown
        # quick solution, gives an okay upperbound on the degree, only works if the degree of the function is less than p-1
        new_deg = min(e*(p-1), d*deg)
        # end of quick solution
        if new_deg >= e*(p-1):
            cells += 1
        else:
            states += floor(new_deg/(p-1))
            new_deg -= states * (p-1)
            if new_deg >= pdiv[-1]:
                states += 1
                if states == e:
                    states = 0
                    cells += 1
            else:
                for subgroup in pdiv:
                    if subgroup > new_deg:
                        break
    print(r, cells, states, subgroup, f"{subgroup+1}*p^{e*cells+states}")

2 0 0 64 65*p^0
3 0 1 128 129*p^1
4 1 0 0 1*p^8
5 1 0 8 9*p^8
6 1 2 0 1*p^10
7 2 0 0 1*p^16
8 2 0 32 33*p^16
9 2 2 64 65*p^18
10 3 0 0 1*p^24
11 3 0 64 65*p^24
12 3 3 0 1*p^27
13 4 0 0 1*p^32
14 4 0 128 129*p^32
15 4 4 8 9*p^36
16 5 0 0 1*p^40
17 5 1 0 1*p^41
18 5 5 128 129*p^45
19 6 0 0 1*p^48
20 6 1 0 1*p^49
21 6 7 128 129*p^55
22 7 0 0 1*p^56
23 7 1 64 65*p^57
24 8 0 0 1*p^64
25 8 0 0 1*p^64


In [42]:
# p-degrees mod p^2
p = 2**8+1
e = 8
d = 7
t = 8

P_cell = get_polytope_affine_power_permutation(p**e, d)
P_round = compose_permutations(t*P_cell, get_polytope_reduced_linear_permutation(p**e, t), 1)
P_full = P_round
r = 1
pdegrees2 = []
while True:
    model = MixedIntegerLinearProgram('PPL', maximization=True)
    digits_input_exponents = model.new_variable(integer=True, nonnegative=True)
    model.set_max(digits_input_exponents, t*(p-1))
    rotations_input_exponents = [sum(p**i*digits_input_exponents[(i-j) % e] for i in range(e)) for j in range(e)]
    valuations = model.new_variable(integer=True, nonnegative=True)
    valuation = 0
    for i in range(e):
        valuation += valuations[i]
        constraint_variables_in_polyhedron(model, P_full, [rotations_input_exponents[i], p**i, valuations[i]])
    model.add_constraint(valuation <= 1)
    model.set_objective(sum(digits_input_exponents[i] for i in range(e)))
    res = model.solve()
    print(r, res)
    pdegrees2.append(res)
    if res >= t*e*(p-1)-1:
        break
    r += 1
    P_full = compose_permutations(P_full, P_round, 1)

1 1799
2 2089
3 2383
4 4097
5 4153
6 4545
7 6148
8 6223
9 6746
10 8200
11 8300
12 8999
13 10254
14 10387
15 11320
16 12309
17 12487
18 13732
19 14367
20 14604
21 16266
22 16383


In [43]:
# properties mod p^2 based on p-degree
pdiv = divisors(p-1)[:-1]
for r, deg in enumerate(pdegrees2, start=1):
    states, subgroup = 0, 0
    states = floor(deg/(p-1))
    deg -= states*(p-1)
    if deg >= pdiv[-1]:
        states += 1
    else:
        for subgroup in pdiv:
            if subgroup > deg:
                break
    print(r, states, subgroup, f"{subgroup+1}*p^{states}")

1 7 8 9*p^7
2 8 64 65*p^8
3 9 128 129*p^9
4 16 2 3*p^16
5 16 64 65*p^16
6 18 0 1*p^18
7 24 8 9*p^24
8 24 128 129*p^24
9 26 128 129*p^26
10 32 16 17*p^32
11 32 128 129*p^32
12 35 64 65*p^35
13 40 16 17*p^40
14 41 0 1*p^41
15 44 64 65*p^44
16 48 32 33*p^48
17 49 0 1*p^49
18 54 0 1*p^54
19 56 32 33*p^56
20 57 16 17*p^57
21 64 0 1*p^64
22 64 0 1*p^64


In [44]:
# SHARK-like properties with higher divisibility based on saturating sboxes
p = 2**8+1
e = 8
d = 7
t = 8

P_cell = get_polytope_affine_power_permutation(p**e, d)
P_round = compose_permutations(t*P_cell, get_polytope_reduced_linear_permutation(p**e, t), 1)
P_full = get_polytope_reduced_linear_permutation(p**e, t)
r = 1
degrees_p2 = []
while True:
    for cells in range(1, t):
        res = 0
        for i in range(e):
            Pred = (P_full & Polyhedron(eqns = [[-p**i, 0, 1, 0], [-(p**e-1)*cells, 1, 0, (p**e-1)]])).linear_transformation(Matrix(ZZ, 1, 3, [[0, 0, 1]]))
            res += min(ceil(x[0]) for x in Pred.vertices_list())
        if res >= 2:
            print(r, cells, res)
    if cells == t-1 and res <= 1:
        break
    r += 1
    P_full = compose_permutations(P_full, P_round, 1)

1 1 8
1 2 8
1 3 16
1 4 16
1 5 24
1 6 24
1 7 32
2 1 8
2 2 8
2 3 8
2 4 8
2 5 8
2 6 8
2 7 8
3 1 8
3 2 8
3 3 8
3 4 8
3 5 8
3 6 8
3 7 8
4 1 7
4 2 8
4 3 8
4 4 8
4 5 8
4 6 8
4 7 8
5 1 7
5 2 7
5 3 7
5 4 7
5 5 7
5 6 7
5 7 7
6 1 7
6 2 7
6 3 7
6 4 7
6 5 7
6 6 7
6 7 7
7 1 6
7 2 7
7 3 7
7 4 7
7 5 7
7 6 7
7 7 7
8 1 6
8 2 6
8 3 6
8 4 6
8 5 6
8 6 6
8 7 6
9 1 6
9 2 6
9 3 6
9 4 6
9 5 6
9 6 6
9 7 6
10 1 5
10 2 5
10 3 6
10 4 6
10 5 6
10 6 6
10 7 6
11 1 5
11 2 5
11 3 5
11 4 5
11 5 5
11 6 5
11 7 5
12 1 5
12 2 5
12 3 5
12 4 5
12 5 5
12 6 5
12 7 5
13 1 4
13 2 4
13 3 4
13 4 5
13 5 5
13 6 5
13 7 5
14 1 4
14 2 4
14 3 4
14 4 4
14 5 4
14 6 4
14 7 4
15 1 4
15 2 4
15 3 4
15 4 4
15 5 4
15 6 4
15 7 4
16 1 3
16 2 3
16 3 3
16 4 3
16 5 4
16 6 4
16 7 4
17 1 3
17 2 3
17 3 3
17 4 3
17 5 3
17 6 3
17 7 3
18 1 3
18 2 3
18 3 3
18 4 3
18 5 3
18 6 3
18 7 3
19 1 2
19 2 2
19 3 2
19 4 2
19 5 2
19 6 3
19 7 3
20 1 2
20 2 2
20 3 2
20 4 2
20 5 2
20 6 2
20 7 2
21 2 2
21 3 2
21 4 2
21 5 2
21 6 2
21 7 2
